In [2]:
import numpy
import time
from tqdm import tqdm
import ipyparallel as ipp
from scipy import stats

from qiskit_nature.operators.second_quantization import FermionicOp
from qiskit_nature.problems.second_quantization.electronic.builders import fermionic_op_builder
from qiskit_nature.mappers.second_quantization import JordanWignerMapper
#from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit.opflow import I,X,Y,Z,StateFn,AbelianGrouper
from qiskit.chemistry import FermionicOperator
from qiskit.providers.aer import AerSimulator
from qiskit.aqua.operators.legacy import op_converter
from openfermion.circuits import slater_determinant_preparation_circuit
from qiskit.quantum_info import Pauli
from qiskit import QuantumCircuit,execute,QuantumRegister,ClassicalRegister
from qiskit import Aer
from openfermion.linalg import givens_decomposition_square as givensRotns
from qiskit.opflow.primitive_ops import PauliSumOp
from qiskit.utils import QuantumInstance
from qiskit.opflow import PauliExpectation, CircuitSampler, CircuitStateFn 
import scipy
from joblib import Parallel, delayed, Memory
from qiskit.aqua.operators import WeightedPauliOperator
from qiskit.compiler import transpile
from qiskit.circuit.random import random_circuit
from qiskit.chemistry.components.variational_forms import UCCSD

In [4]:
import warnings
warnings. filterwarnings('ignore')

In [2]:
c =ipp.Client()
num_qubits=8

NameError: name 'ipp' is not defined

In [7]:
def qubitOp(h1,h2):
    qubit_op=FermionicOperator(h1,h2).mapping('jordan_wigner')
    return qubit_op

In [8]:
def egBandHamiltonian(U):
    #Getting chemical Potential for Half-Filling
    with open('../chem_pot_for_Half_Fill.txt','r') as f:
        lines=f.readlines()[1:]
        for line in lines:
            elems=line.split()
            if int(elems[0])==U:
                muHalf=float(elems[1]) #Chem Pot for a given Hubbard U
    #Getting the one body and two body interaction vertexes
    with open('../'+str(U)+'/v1e.dat','r') as f:
            lines=f.readlines()[1:]
            num_sites=4
            chem_pot=numpy.zeros((2*num_sites,2*num_sites))
            eg_h1=numpy.zeros((2*num_sites,2*num_sites))
            for line in lines:
                elems=line.split()
                eg_h1[int(elems[0])][int(elems[1])]=float(elems[2])
                eg_h1[int(elems[0])+num_sites][int(elems[1])+num_sites]=float(elems[2])
            for i in range(2*num_sites):
                chem_pot[i][i]=-muHalf
            eg_h1=eg_h1+chem_pot       
    with open('../'+str(U)+'/v2e.dat','r') as f:
        num_sites=4
        eg_h2=numpy.zeros((2*num_sites,2*num_sites,2*num_sites,2*num_sites))
        for line in f:
            if "#" in line:
                continue
            line = line.split()
            i,j,k,l = map(int, line[:4])
            val = float(line[4])
            eg_h2[i,j,k,l] = eg_h2[i+num_sites,j+num_sites,k,l] = eg_h2[i,j,k+num_sites,l+num_sites] \
            = eg_h2[i+num_sites,j+num_sites,k+num_sites,l+num_sites] = 0.5*val  # convention with 0.5 factor included.
    qubitH=qubitOp(eg_h1,eg_h2)
    Hmat=op_converter.to_matrix_operator(qubitH).dense_matrix
    w,v=numpy.linalg.eigh(Hmat)
    Eg=w[0]
    state_g=v[:,0]
    return qubitH,Hmat,Eg,state_g

In [9]:
%run Utils.ipynb
%run optimizers.ipynb
%run EstimateHamiltonianMean.ipynb

In [10]:
U=7
qubitH,Hmat,Eg,state_g=egBandHamiltonian(U)

In [1]:
#Hamiltonian grouping for circuit based measurement
eg_h1_0,eg_h1_1,eg_h2_0,eg_h2_1=egBandHamiltonianPartition(U)
qubitH_0,qubitH_1,qubitH_2=qubitHamiltonianGroups(U)
opArr=[qubitH_0,qubitH_1,qubitH_2]
#two body off diags
circ0_1gate,rotH2body=OffDiag2BodyTerms(opArr[2])
#one body off diags
circ0_8gate,rotH1body=OffDiag1BodyTerms(opArr[0])
Ops=[rotH1body,opArr[1],rotH2body]

NameError: name 'egBandHamiltonianPartition' is not defined

In [12]:
#Control Hamiltonian at lower U value
eg_h1_0,eg_h1_1,eg_h2_0,eg_h2_1=egBandHamiltonianPartition(U-3)
qubitH_0,qubitH_1,qubitH_2=qubitHamiltonianGroups(U-3)
opArr=[qubitH_0,qubitH_1,qubitH_2]
#two body off diags
circ0_1gate_ctrl,rotH2body=OffDiag2BodyTerms(opArr[2])
#one body off diags
circ0_8gate_ctrl,rotH1body=OffDiag1BodyTerms(opArr[0])
Ops_ctrl=[rotH1body,opArr[1],rotH2body]

In [13]:
#commutator pools
commutator_pool,commutator_pool_2=commutatorPool(qubitH)

In [14]:
U=7
backend=Aer.get_backend('statevector_simulator')
HFcirc,HFstate=HF_init_state(U)
var_form_base=UCCSD(8,num_particles=4, initial_state=HFcirc,qubit_mapping='jordan_wigner',two_qubit_reduction=False)
var_form_base.manage_hopping_operators()

Final Hartree Fock Energy -18.788335371650774


In [37]:
for i in range(len(ExcOps)):
    var_form_base.push_hopping_operator(WeightedPauliOperator([[1j,Pauli.from_label(ExcOps[i])]]))

In [16]:
runs=120
tol=5e-6
qr=QuantumRegister(8,'q')

In [39]:
#params=[]
EnergyArr=[]
numCX=0
CXarr=[]
#ExcOps=[]
error=1000
steps=33
stepNo=7
timeArr=[]
t1=time.time()
num_shots_arr=[2**14,2**15,2**15]
while error>1e-5 and stepNo<steps:
    PauliOp=ansatzExpander(ordChoice='1')
    ExcOps.append(PauliOp.paulis[0][1].to_label())
    print("excitationOp",ExcOps[-1])
    with open('../7/labels.txt','+a') as f:
        print(ExcOps[-1],file=f)
    params.append(0.0)
    var_form_base.push_hopping_operator(PauliOp)
    #construct circuit for estimating Hamiltonian
    circ=var_form_base.construct_circuit(parameters=params)
    params,E=AdaDelta_qasm(HamiltonianEstmFrugal,params,num_shots_arr=num_shots_arr,runs=runs,tol=tol,save_opt_steps=True)
#    params,E=AdaDelta_sv(Energy,params,runs=runs,tol=tol,save_opt_steps=True)
#    params,E=SMO_qasm(HamiltonianEstmFrugal,params,runs=runs,tol=tol,save_opt_steps=True) 
#    params,E=AdaDeltaSMO_qasm(HamiltonianEstmFrugal,params,num_shots_arr=[2**8,2**9,2**9],runs=runs,tol=tol,save_opt_steps=True)
    params=list(params)
    numCX=numCX+countCXs(ExcOps[-1])
    CXarr.append(numCX)
    EnergyArr.append(E)
    timeArr.append(time.time()-t1)
    error=E-Eg
    print("convergence energy",EnergyArr[-1])
    
    print("time elapsed",timeArr[-1])
    stepNo=stepNo+1
    
    

[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  27 tasks      | elapsed:   44.5s


max. gradient 0.22576366559004002
excitationOp IIXYIIXX


[Parallel(n_jobs=7)]: Done  56 out of  56 | elapsed:   44.8s finished


index 0
updated energy -19.154552229039176 learning rate [0.99659413+0.j 0.47029897+0.j 0.45183394+0.j 0.86246082+0.j
 0.47974966+0.j 0.74435477+0.j 0.64643775+0.j 0.57159255+0.j
 0.55276399+0.j 0.49539083+0.j 0.95122356+0.j 0.61988805+0.j
 0.31241228+0.j 0.25782368+0.j 0.15121384+0.j]
error 0.7516566981132087
index 1
updated energy -19.155262192597526 learning rate [0.9938273 +0.j 0.55883132+0.j 0.16679272+0.j 0.43671846+0.j
 0.37886232+0.j 0.13090269+0.j 0.37429572+0.j 0.21289793+0.j
 0.58051849+0.j 0.28721481+0.j 0.29644924+0.j 0.15366491+0.j
 0.45055845+0.j 0.23641303+0.j 0.1894349 +0.j]
error 0.563920014474494
index 2
updated energy -19.146473018696707 learning rate [0.86046842+0.j 0.61463919+0.j 0.1624028 +0.j 0.52612963+0.j
 0.48532564+0.j 0.18895372+0.j 0.12473528+0.j 0.28503951+0.j
 0.24107772+0.j 0.25818063+0.j 0.31569198+0.j 0.20843747+0.j
 0.18339322+0.j 0.15019936+0.j 0.22052842+0.j]
error 0.4251373043310753
index 3
updated energy -19.14293097558957 learning rate [0.834717

KeyboardInterrupt: 

In [ ]:
# E_arr=[]
# E_arr2=[]
# E_arr3=[]
# num_shots_arr=[2**8,2**9,2**9]
# string_shots_label=','.join([str(shots) for shots in num_shots_arr])
# with open('../'+str(U)+'/adaDeltaOptStepsWithQasm'+string_shots_label+'.txt','r') as f:
#     lines=f.readlines()
#     for line in lines:
#         elems=line.split('#')
#         E_arr.append(float(elems[1]))
# num_shots_arr=[2**4,2**5,2**5]
# string_shots_label=','.join([str(shots) for shots in num_shots_arr])
# with open('../'+str(U)+'/adaDeltaOptStepsWithQasm'+string_shots_label+'.txt','r') as f:
#     lines=f.readlines()
#     for line in lines:
#         elems=line.split('#')
#         E_arr2.append(float(elems[1]))
# num_shots_arr=[2**5,2**6,2**6]
# string_shots_label=','.join([str(shots) for shots in num_shots_arr])
# with open('../'+str(U)+'/adaDeltaOptStepsWithQasm'+string_shots_label+'.txt','r') as f:
#     lines=f.readlines()
#     for line in lines:
#         elems=line.split('#')
#         E_arr3.append(float(elems[1]))        

In [ ]:
# import matplotlib.pyplot as plt
# figure,ax=plt.subplots(1,figsize=(10,7))
# ax.set_yscale('log')
# ax.tick_params(labelsize=15)
# ax.plot(numpy.array(E_arr[:363])+19.0701686065437848,label='shots=2^8,2^9,2^9',marker='o')
# ax.set_xlabel('runs')
# ax.legend(fontsize=15)
# ax.set_ylabel('abs. Error')